## Output and Suggestions Test Design 2

This notebook uses the Google Generative AI (genai) model to grade an essay and suggest improvements. The output shows Gemini's revised essay, with highlighted changes and hoverable tooltips. The tooltip contains the criterion from the rubric that needs to be improved, the improvement, and reasoning for the improvement.

In [1]:
# Import Libraries
import pathlib
import textwrap
import os
import google.generativeai as genai
from IPython.display import display, Markdown, HTML
from dotenv import load_dotenv, find_dotenv
import json
import re


In [2]:
# Load environment variables
load_dotenv(find_dotenv())
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Configure the API client with the API key
genai.configure(api_key=GOOGLE_API_KEY)


In [3]:
# Utility Functions

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def upload_rubric(file_path):
    if file_path is None:
        raise ValueError("No rubric file path provided.")
    with open(file_path, 'r') as file:
        content = file.read()
    return content


In [4]:
# Prompt Formatting and Essay Analysis

def format_prompt(rubric_content, essay_text):
    """
    Formats the prompt for the model to validate and grade the essay.
    Combines the rubric content and the student essay into a single prompt for the AI model.
    """
    return f"""
    Validate the following grading rubric for formatting and appropriateness:

    Grading Rubric:
    {rubric_content}

    Is the rubric formatted correctly and appropriate for grading the essay? (yes or no) Please explain why for either case. 
    If no, please stop. If yes, grade the essay.

    Student Response:
    {essay_text}

    Output Format:

    ### Grading Rubric and Feedback
    | Criterion | Score | Explanation |
    |-----------|-------|-------------|
    | Criterion | Score | Explanation |
    | Criterion | Score | Explanation |
    | Criterion | Score | Explanation |
    | Criterion | Score | Explanation |
    | Criterion | Score | Explanation |
    
    Please do not say criterion and the criterion number next to the actual criterion. Always provide an explanation.
        
    Please make the final score a weighted average of all the scores. If the rubric has weights in the criterion section, please take that into account.
    """

def analyze_essay_with_rubric(model, rubric_content, essay_text):
    """
    Sends the formatted prompt to the model for grading.
    Uses the AI model to analyze the essay based on the provided rubric.
    """
    prompt = format_prompt(rubric_content, essay_text)
    response = model.generate_text(prompt=prompt)
    return response.result


In [5]:
# Suggestions and Essay Rewriting

def format_suggestions_and_rewrite_prompt(grading_feedback, essay_text):
    """
    Formats the prompt for generating suggestions and rewriting the essay.
    Combines the grading feedback and the original essay into a single prompt for the AI model.
    """
    return f"""
    Based on the following grading feedback, provide suggestions for improvement and rewrite the essay using the improvements.

    Grading Feedback:
    {grading_feedback}

    Essay:
    {essay_text}

    Take each improvement provided and identify the smallest chunk of the original text where it was applied, and the part of the rubric.
    Provide the improvements in the following JSON format: 
    [
        {{"improvement": "improvement_1", "criterion_from_rubric": "criterion_from_rubric_1", "reason_for_suggestion": "reason_for_suggestion_1", "original_text": "original_text_1", "revised_text": "revised_text_1"}},
        ...
        {{"improvement": "improvement_n", "criterion_from_rubric": "criterion_from_rubric_n", "reason_for_suggestion": "reason_for_suggestion_n", "original_text": "original_text_n", "revised_text": "revised_text_n"}}
    ]
    """

def generate_suggestions_and_rewrite_essay(model, grading_feedback, essay_text):
    """
    Sends the formatted prompt to the model for suggestions and essay rewrite.
    Uses the AI model to generate suggestions for improving the essay and then rewrites it based on those suggestions.
    """
    prompt = format_suggestions_and_rewrite_prompt(grading_feedback, essay_text)
    response = model.generate_text(prompt=prompt)
    if response and response.result:
        print("Full response from model:\n", response.result)  # Debugging statement
        return response.result
    else:
        print("Error: No response from model")
        return None


In [6]:
# JSON Cleaning

def clean_json_string(json_string):
    """
    Cleans the JSON string for parsing.
    Removes any extraneous characters or formatting issues that may prevent JSON parsing.
    """
    json_string = json_string.replace('```json', '').replace('```', '').strip()
    json_string = re.sub(r',\s*([}\]])', r'\1', json_string)  # Remove trailing commas
    json_string = re.sub(r'\s*}\s*{', '},{', json_string)  # Ensure all objects are closed properly
    if json_string.count('[') > json_string.count(']'):
        json_string += ']'
    if json_string.count('{') > json_string.count('}'):
        json_string += '}'
    return json_string


In [7]:
# Extract JSON Improvements

def extract_json_improvements(response):
    """
    Extracts and cleans JSON improvements from the model response.
    Parses the model's response to extract improvements in JSON format and returns them along with the response without JSON.
    """
    if response is None:
        print("Error: No response provided for JSON extraction")
        return {'improvements': [], 'response_without_json': ''}
    try:
        json_match = re.search(r'\[\s*\{.*\}\s*\]', response, re.DOTALL)
        if not json_match:
            raise ValueError("No valid JSON found in response.")
        
        cleaned_json_string = clean_json_string(json_match.group(0))
        print("Cleaned JSON string:", cleaned_json_string)

        improvements = json.loads(cleaned_json_string)
        response_without_json = response.replace(json_match.group(0), '')
        return {'improvements': improvements, 'response_without_json': response_without_json}
    except (ValueError, json.JSONDecodeError) as e:
        print("Error extracting JSON from response:", e)
        print(f"Failed to decode JSON string: {response}")
        return {'improvements': [], 'response_without_json': response}
    

In [8]:
# Display Improvements

def display_improvements(improvements, original_text):
    """
    Displays the improvements in the original essay text with hover effects.
    Highlights the revised text in the essay and provides a tooltip with the details of the improvement.
    """
    if not improvements:
        print("No improvements to display.")
        return

    css = """
    <style>
    .improvement-tooltip {
        position: relative;
        display: inline-block;
        cursor: pointer;
        background-color: yellow;
    }
    .improvement-tooltip .tooltiptext {
        visibility: hidden;
        width: 300px;
        background-color: white;
        color: black;
        text-align: left;
        border: 1px solid #ddd;
        padding: 10px;
        border-radius: 6px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        position: absolute;
        z-index: 1;
        top: 100%;
        left: 50%;
        margin-left: -150px;
    }
    .improvement-tooltip:hover .tooltiptext {
        visibility: visible;
    }
    </style>
    """
    
    html_content = original_text
    
    for improvement in improvements:
        original = improvement.get('original_text', '')
        revised = improvement.get('revised_text', '')
        improvement_text = improvement['improvement']
        reason_for_suggestion = improvement['reason_for_suggestion']
        criterion = improvement['criterion_from_rubric']
        
        tooltip_html = f"""
        <span class="improvement-tooltip">
            {revised}
            <span class="tooltiptext">
                <strong>Criterion:</strong> {criterion}<br>
                <strong>Improvement:</strong> {improvement_text}<br>
                <strong>Reason:</strong> {reason_for_suggestion}
            </span>
        </span>
        """
        
        if original:
            html_content = html_content.replace(original, tooltip_html)
        else:
            html_content = html_content + tooltip_html
    
    display(HTML(css + html_content))


In [9]:
# Rubric Selection

def list_rubrics(rubric_folder):
    return [f for f in os.listdir(rubric_folder) if f.endswith('.md')]

def select_rubric(rubric_folder):
    """
    Allows user to select a rubric from the list.
    Lists all available rubric files in the specified folder and prompts the user to select one by entering its corresponding number.
    """
    rubrics = list_rubrics(rubric_folder)
    print("Available rubrics:")
    for idx, rubric in enumerate(rubrics, 1):
        print(f"{idx}. {rubric}")
    
    rubric_index = int(input("Select a rubric by entering the corresponding number: ")) - 1
    selected_rubric = rubrics[rubric_index]
    return os.path.join(rubric_folder, selected_rubric)

In [10]:
# Define the relative path to the rubric folder from the current directory
current_dir = os.getcwd()
relative_rubric_folder = '../nys_common_core_derived'
rubric_folder = os.path.join(current_dir, relative_rubric_folder)

# Load the Rubric
if not os.path.exists(rubric_folder):
    print(f"The directory {rubric_folder} does not exist. Creating the directory.")
    os.makedirs(rubric_folder)
    print(f"Please add rubric files in markdown format (.md) to the {rubric_folder} directory and re-run the notebook.")
else:
    rubric_files = list_rubrics(rubric_folder)
    if not rubric_files:
        print(f"No rubric files found in the directory {rubric_folder}. Please add rubric files in markdown format (.md) and re-run the notebook.")
    else:
        rubric_file_path = select_rubric(rubric_folder)
        if rubric_file_path:
            rubric_content = upload_rubric(rubric_file_path)
            display(to_markdown(rubric_content))


Available rubrics:
1. science_rubric_9_12.md
2. writing_ela_rubric_11_12.md
3. writing_ela_rubric_9_10.md
4. writing_social_sciences_11_12.md
Select a rubric by entering the corresponding number: 4


> 
> # New York State Common Core Writing Standards Rubric for Social Sciences (Grades 11-12)
> 
> | Criterion                  | 1 | 2 | 3 | 4 | 5 |
> |----------------------------|---|---|---|---|---|
> | **1. Argument Development** | Introduces claims with little or no clarity; minimal distinction between claims and counterclaims; lacks logical organization | Introduces claims with some clarity; some distinction between claims and counterclaims; basic organization but lacks depth | Introduces clear claims; distinguishes claims from counterclaims; logical sequence but may have minor gaps | Introduces precise and knowledgeable claims; clearly distinguishes claims from counterclaims; logically sequences with minor issues | Introduces precise, knowledgeable claims; clearly distinguishes and logically sequences claims, counterclaims, reasons, and evidence |
> | **2. Evidence and Analysis** | Provides minimal or irrelevant evidence; weak development of claims and counterclaims; lacks consideration of audience | Provides some relevant evidence; basic development of claims and counterclaims; limited audience consideration | Provides relevant evidence and analysis; adequately develops claims and counterclaims; considers audience to some extent | Provides strong evidence and analysis; thoroughly develops claims and counterclaims; anticipates audience’s knowledge and biases | Provides the most relevant evidence and analysis; thoroughly develops claims and counterclaims with strong audience consideration |
> | **3. Cohesion and Clarity** | Minimal use of transitions and varied syntax; weak cohesion and unclear relationships between ideas | Uses some transitions and varied syntax; some cohesion but relationships between ideas may be unclear | Uses appropriate transitions and varied syntax; clear relationships between most ideas | Uses varied syntax and transitions effectively; clear and cohesive relationships between ideas | Uses varied syntax and transitions skillfully; creates strong cohesion and clear relationships between all ideas |
> | **4. Style and Tone** | Inconsistent or inappropriate style and tone; minimal adherence to discipline norms | Somewhat consistent style and tone; some adherence to discipline norms | Consistent style and tone; adheres to discipline norms but may have minor lapses | Maintains formal style and objective tone; adheres to discipline norms consistently | Establishes and maintains formal style and objective tone; fully adheres to discipline norms |
> | **5. Conclusion** | Provides a weak or irrelevant conclusion; does not support the argument | Provides a basic conclusion; somewhat supports the argument | Provides a clear conclusion; supports the argument but may lack depth | Provides a strong conclusion; effectively supports the argument | Provides a compelling conclusion; thoroughly supports the argument and adds insight |
> 


In [11]:
# Essay Text
essay_text = """
The American Revolution, which occurred from 1775 to 1783, was a pivotal event in history that not only led to the birth of a new nation but also had profound and lasting impacts on modern American society. This essay explores the significance of the American Revolution, examining its influence on contemporary political systems, social structures, and cultural values in the United States.

Firstly, the American Revolution laid the foundation for the modern American political system. The revolutionaries' fight for independence was rooted in the desire for self-governance and the rejection of tyrannical rule. This struggle culminated in the drafting of the Declaration of Independence in 1776, which articulated the fundamental principles of liberty, equality, and democracy. These principles became the cornerstone of the United States Constitution, adopted in 1787, which established a federal republic with a system of checks and balances designed to prevent the concentration of power. Today, these democratic ideals continue to shape American politics, ensuring that power is derived from the consent of the governed and that individual rights are protected.

Moreover, the American Revolution had a profound impact on the social fabric of the nation. The revolution challenged existing social hierarchies and promoted the idea of equality. While the promise of equality was not immediately realized for all groups, the rhetoric of the revolution inspired subsequent movements for social justice. The abolitionist movement, which sought to end slavery, drew upon the revolutionary principles of liberty and equality. Similarly, the women's suffrage movement, which fought for women's right to vote, was influenced by the revolution's emphasis on individual rights. These movements have played a crucial role in shaping a more inclusive and equitable society, reflecting the enduring legacy of the American Revolution.

Culturally, the American Revolution fostered a sense of national identity and unity. The shared struggle for independence created a collective memory and a sense of common purpose among the American people. This sense of unity was further reinforced by the creation of national symbols, such as the flag and the national anthem, which continue to evoke patriotic sentiments. Additionally, the revolution gave rise to a distinct American culture that valued individualism, self-reliance, and innovation. These cultural values have profoundly influenced various aspects of American life, including the economy, education, and popular culture, contributing to the country's dynamic and entrepreneurial spirit.

In conclusion, the American Revolution was a transformative event that has had a lasting impact on modern American society. It established the foundational principles of American democracy, inspired social movements for equality, and fostered a unique national identity and culture. The legacy of the American Revolution continues to resonate in contemporary America, shaping its political systems, social structures, and cultural values. As Americans reflect on their history, the revolutionary ideals of liberty, equality, and democracy remain guiding principles for the nation's ongoing pursuit of a more just and equitable society.
"""


In [12]:
# Validate and Grade the Essay
grading_feedback = analyze_essay_with_rubric(genai, rubric_content, essay_text)
print("Grading Feedback:", grading_feedback)
display(Markdown(grading_feedback))


Grading Feedback: **Grading Rubric and Feedback**

| Criterion | Score | Explanation |
|---|---|---|
| Argument Development | 4 | Introduces clear claims; distinguishes claims from counterclaims; logical sequence with minor issues |
| Evidence and Analysis | 4 | Provides relevant evidence and analysis; adequately develops claims and counterclaims; considers audience to some extent |
| Cohesion and Clarity | 4 | Uses appropriate transitions and varied syntax; clear relationships between most ideas |
| Style and Tone | 4 | Maintains formal style and objective tone; adheres to discipline norms consistently |
| Conclusion | 4 | Provides a clear conclusion; supports the argument but may lack depth |
| **Final Score** | 4.0 |

The essay is well-written and demonstrates a strong understanding of the topic. The author clearly introduces the claims and counterclaims, provides relevant evidence and analysis, and maintains a formal style and tone. The essay could be improved by providing more dep

**Grading Rubric and Feedback**

| Criterion | Score | Explanation |
|---|---|---|
| Argument Development | 4 | Introduces clear claims; distinguishes claims from counterclaims; logical sequence with minor issues |
| Evidence and Analysis | 4 | Provides relevant evidence and analysis; adequately develops claims and counterclaims; considers audience to some extent |
| Cohesion and Clarity | 4 | Uses appropriate transitions and varied syntax; clear relationships between most ideas |
| Style and Tone | 4 | Maintains formal style and objective tone; adheres to discipline norms consistently |
| Conclusion | 4 | Provides a clear conclusion; supports the argument but may lack depth |
| **Final Score** | 4.0 |

The essay is well-written and demonstrates a strong understanding of the topic. The author clearly introduces the claims and counterclaims, provides relevant evidence and analysis, and maintains a formal style and tone. The essay could be improved by providing more depth in the conclusion.

In [13]:
# Generate Suggestions and Rewrite the Essay
suggestions_and_rewrite = generate_suggestions_and_rewrite_essay(genai, grading_feedback, essay_text)
extracted_data = extract_json_improvements(suggestions_and_rewrite)
print("Extracted Data:", extracted_data)


Full response from model:
 [
        {"improvement": "Add a clear thesis statement", "criterion_from_rubric": "Argument Development", "reason_for_suggestion": "The essay lacks a clear thesis statement that introduces the main argument.", "original_text": "", "revised_text": "The American Revolution had a profound and lasting impact on modern American society, shaping its political systems, social structures, and cultural values."},
        {"improvement": "Provide more evidence and analysis", "criterion_from_rubric": "Evidence and Analysis", "reason_for_suggestion": "The essay provides some evidence and analysis, but it could be more extensive.", "original_text": "The revolutionaries' fight for independence was rooted in the desire for self-governance and the rejection of tyrannical rule.", "revised_text": "The revolutionaries' fight for independence was rooted in the desire for self-governance and the rejection of tyrannical rule. This struggle culminated in the drafting of the Declar

In [14]:
# Display the Suggestions and Revised Essay
display(Markdown(f"**Suggestions and Revised Essay:**\n\n{extracted_data['response_without_json']}"))


**Suggestions and Revised Essay:**



In [15]:
# Display the Improvements in the New Essay with Hover Effect
display_improvements(extracted_data['improvements'], essay_text)
